In [1]:
import nglview
view = nglview.show_pdbid("1fjs")  # load "3pqr" from RCSB PDB and display viewer widget
#view.add_surface(selection="protein", color="blue")
view.background = 'black'
view.color_by('atomindex', component=1)
view

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [10]:
?view.add_surface

In [22]:
view.color_by('atomindex', component=1)